# 0. Imports and constants (run config)

In [1]:
from copy import deepcopy
from datetime import datetime

from copulae.archimedean import ClaytonCopula, FrankCopula, GumbelCopula
from copulae.elliptical import GaussianCopula, StudentCopula
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline

from components.transformers.rate_transformer import RateTransformer
from components.transformers.log_transformer import LognTransformer
from components.wrappers.kde_wrapper import KernelDensityWrapper
from components.evaluate.cv.monthly_splitters import ExpandingMonthlySplitter, SlidingMonthlySplitter
from components.evaluate.evaluate import evaluate

In [2]:
RANDOM_SEED = 5

SELECTED_PORTFOLIOS = {
    'P1': ['Nickel', 'Copper'],
    'P2': ['Brent Oil', 'Gas US'],
    'P3': ['Gold', 'Silver'],
#     'P4': ['Nickel', 'Copper', 'Gold'],
#     'P5': ['Nickel', 'Copper', 'Silver'],
#     'P6': ['Brent Oil', 'Gas US', 'Gold'],
#     'P7': ['Brent Oil', 'Gas US', 'Silver']
}

SELECTED_BENCHMARK = 'SPGSCI'

# KDE configuration
# 'gaussian', 'tophat', 'epanechnikov', 'exponential', 'linear', 'cosine',
KERNEL = 'epanechnikov'
# 'silverman', 'scott', float
KERNEL_BANDWIDTH = 'silverman'

# 'gaussian', 'student', 'frank', 'clayton', 'gumbel'
COPULA = 'student'
# Initiate objects
# Copula
copula_dict = {
    'gaussian': GaussianCopula,
    'student': StudentCopula,
    'frank': FrankCopula,
    'clayton': ClaytonCopula,
    'gumbel': GumbelCopula
}

# Cross validation configuration
# expanding window
#CV = ExpandingMonthlySplitter(initial_window=150, step_length=1, test_horizon=1)
# sliding window
CV = SlidingMonthlySplitter(initial_window=130, step_length=1, test_horizon=1, window_length=24)

# 1. Data load and prep

In [3]:
# load prices from dataset
df = pd.read_excel('data/BazaMSA.xlsx', sheet_name='Ceny')
# Dates as index
df.set_index('Dates', inplace=True)

# Select only necessary columns
assets_in_portfolios = list(set([i for j in SELECTED_PORTFOLIOS.values() for i in j]))
assets_in_portfolios.append(SELECTED_BENCHMARK)
df = df[assets_in_portfolios]

# Comprehensive scikitlearn `Pipeline` that first creates the rates
# and then performs logn on top
log_rate_pipe = Pipeline(
    steps=[
        ('rates', RateTransformer(period=1)),
        ('logn', LognTransformer())
    ]
)
df_transformed = log_rate_pipe.fit_transform(X=df)
df_transformed.dropna(inplace=True)

# 2. Run evaluation

In [4]:
results_df = pd.DataFrame()
for portfolio, assets in SELECTED_PORTFOLIOS.items():

    X = df_transformed[assets]
    X_benchmark = df_transformed[SELECTED_BENCHMARK]

    portfolio_results_df = evaluate(
        X=X,
        X_benchmark=X_benchmark,
        kernel=KernelDensityWrapper(kernel=KERNEL, bandwidth=KERNEL_BANDWIDTH),
        copula=copula_dict[COPULA](dim=len(assets)),
        cv=CV,
        random_seed=RANDOM_SEED,
        n_copula_simulations=X.shape[0]
    )
    portfolio_results_df['Portfolio'] = portfolio
    portfolio_results_df['Assets'] = [assets for _ in range(0, len(portfolio_results_df))]
    
    results_df = pd.concat([results_df, portfolio_results_df], ignore_index=True)

col_order = ['Portfolio', 'Assets']
col_order.extend(portfolio_results_df.columns[:-2])

results_df = results_df[col_order]

           Nickel      Copper
count  519.000000  519.000000
mean     0.502781    0.503249
std      0.288395    0.288298
min      0.000106    0.000131
25%      0.254099    0.251073
50%      0.490088    0.494509
75%      0.755550    0.751390
max      1.000000    1.000000
           Nickel      Copper
count  519.000000  519.000000
mean     0.502805    0.503163
std      0.288417    0.288376
min      0.000107    0.000131
25%      0.252187    0.247280
50%      0.489382    0.493700
75%      0.754232    0.747650
max      1.000000    1.000000
           Nickel      Copper
count  521.000000  521.000000
mean     0.503053    0.503396
std      0.288308    0.288300
min      0.000117    0.000127
25%      0.254334    0.246422
50%      0.497688    0.491612
75%      0.756308    0.749516
max      1.000000    1.000000
           Nickel      Copper
count  521.000000  521.000000
mean     0.502984    0.503332
std      0.288273    0.288301
min      0.000114    0.000127
25%      0.252267    0.249125
50%      0

In [5]:
results_condensed = deepcopy(results_df)
results_condensed['Markowitz weights'] = [np.around(i, 5) for i in results_condensed['Markowitz weights'].values]
results_condensed['Semiparametric weights'] = [np.around(i, 5) for i in results_condensed['Semiparametric weights'].values]
results_condensed

,Portfolio,Assets,Train start,Train end,Test start,Test end,Markowitz weights,Semiparametric weights,Markowitz return,Semiparametric return,Benchmark return,Markowitz volatility,Semiparametric volatility,Benchmark volatility,Markowitz sharpe,Semiparametric sharpe
0,P1,"[Nickel, Copper]",2019-02-01,2021-01-31,2021-02-01,2021-02-28,"[0.22529, 0.77471]","[0.20447, 0.79553]",0.006380,0.006486,0.005070,0.013930,0.014001,0.010205,0.094071,0.101148
1,P1,"[Nickel, Copper]",2019-03-01,2021-02-28,2021-03-01,2021-03-31,"[0.24201, 0.75799]","[0.2206, 0.7794]",-0.002824,-0.002725,-0.000881,0.014220,0.014103,0.016518,-0.136601,-0.130717
2,P1,"[Nickel, Copper]",2019-04-01,2021-03-31,2021-04-01,2021-04-30,"[0.22805, 0.77195]","[0.21583, 0.78417]",0.004934,0.004942,0.003592,0.011566,0.011578,0.009198,0.116000,0.116605
3,P1,"[Nickel, Copper]",2019-05-01,2021-04-30,2021-05-01,2021-05-31,"[0.23042, 0.76958]","[0.21855, 0.78145]",0.001792,0.001802,0.001243,0.014365,0.014366,0.011358,0.038229,0.038934
4,P1,"[Nickel, Copper]",2019-06-01,2021-05-31,2021-06-01,2021-06-30,"[0.22736, 0.77264]","[0.22089, 0.77911]",-0.003140,-0.003169,0.001443,0.015532,0.015556,0.009117,-0.295053,-0.296460
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,P3,"[Gold, Silver]",2020-11-01,2022-10-31,2022-11-01,2022-11-30,"[1.0, 0.0]","[1.0, 0.0]",0.003149,0.003149,-0.001137,0.010720,0.010720,0.014673,0.399839,0.399839
74,P3,"[Gold, Silver]",2020-12-01,2022-11-30,2022-12-01,2022-12-31,"[1.0, 0.0]","[1.0, 0.0]",0.001808,0.001808,-0.000815,0.010338,0.010338,0.014239,0.253741,0.253741
75,P3,"[Gold, Silver]",2021-01-01,2022-12-31,2023-01-01,2023-01-31,"[1.0, 0.0]","[1.0, 0.0]",0.002558,0.002558,-0.000300,0.006238,0.006238,0.013479,0.458238,0.458238
76,P3,"[Gold, Silver]",2021-02-01,2023-01-31,2023-02-01,2023-02-28,"[1.0, 0.0]","[1.0, 0.0]",-0.002668,-0.002668,-0.002037,0.006875,0.006875,0.011987,-0.091844,-0.091844


# 3. Save to excel


In [6]:
results_condensed.to_excel(f"results_{datetime.now().strftime('%d_%m_%Y_%H_%M_%S')}.xlsx", index=False)